### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv("api_keys.env")
nyt_api_key = os.getenv("NYT_API_KEY")
print(nyt_api_key)

### Access the New York Times API

In [ ]:
# Set the base URL
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
# headline should be "love"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
api_key = nyt_api_key
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
sort = "newest"
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"
begin_date = "20130101"
end_date = "20230531"
query_url = f"{base_url}api-key={api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date }"

In [ ]:
# Create an empty list to store the reviews
movie_reviews_list = []

# loop through pages 0-19
for page in  range (0,5):
    
    # create query with a page number
    # API results show 10 articles at a time
    page_query = f"{query_url}&page={page}" 
    
    # Make a "GET" request and retrieve the JSON
    movie_reviews= requests.get(page_query).json()
    print(movie_reviews)
    print(movie_reviews.get("response", {}).get("docs", []))

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        for reviews in movie_reviews["response"]["docs"]:
            movie_reviews_list.append(reviews)
            
        # loop through the reviews["response"]["docs"] and append each review to the list
        # Print the page that was just retrieved
        print("Checked page:", page)
        # Print the page number that had no results then break from the loop
    except KeyError:
        print("Error: Key 'response' or 'docs' not found in the API response.")
        break  # Exit the loop if there's an error

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

try:
    for i in range(5):
        print(json.dumps(movie_reviews_list[i], indent=4))
except IndexError:
    print("Error: Index out of range.")

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
movie_reviews_df = pd.json_normalize(movie_reviews_list)
movie_reviews_df


In [ ]:

# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early


def extract_title(col):
    start = col.find('\u2018') + 1
    end = col.find(' 2019 Review', start)
    return col[start:end]



movie_reviews_df['title'] = movie_reviews_df['headline.main'].apply(extract_title)
movie_reviews_df.head()

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
# Extract 'name' and 'value'
# Append the keyword item to the extracted_keywords list
# Fix the "keywords" column by converting cells from a list to a string
movie_reviews_df = {'keywords': [[{'name': 'topic', 'value': 'AI'}, {'name': 'subject', 'value': 'Machine Learning'}],
                                    [{'name': 'topic', 'value': 'Data Science'}, {'name': 'subject', 'value': 'Python'}]]}

extracted_keywords = []
if isinstance(movie_reviews_df, pd.DataFrame) and 'keywords' in movie_reviews_df.columns:
    for index, row in movie_reviews_df.iterrows():
        keywords_list = row['keywords']
        for keyword_item in keywords_list:
            extracted_keywords.append({'name': keyword_item['name'], 'value': keyword_item['value']})

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = []
if isinstance(movie_reviews_df, pd.DataFrame):
    if 'title' in movie_reviews_df.columns:
        print("Before creating titles_list")
        titles_list = movie_reviews_df['title'].to_list()
        print("After creating titles_list")
print(titles_list)


### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep after every 50 requests
request_counter = 0

# Loop through the titles
for title in titles_list[:5]:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Reached a multiple of 50 requests. Sleeping for 10 seconds.")
        time.sleep(10)

    # Add 1 to the request counter
    request_counter += 1
    if request_counter >= 100:
        break

    # Perform a "GET" request for The Movie Database
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        response = requests.get(url + title + tmdb_api_key)
        if response.status_code == 200:
            movie_data = response.json()

            # Extract IMDb ID (TMDb ID) from the search results
            if "id" in movie_data:
                imdb_id = movie_data["id"]
                print(f"IMDb ID for '{title}': {imdb_id}")
            else:
                print(f"IMDb ID not found for '{title}'")

            # Extract relevant information (e.g., title, release date, genres)
            movie_info = {
                "title": movie_data.get("title"),
                "release_date": movie_data.get("release_date"),
                "genres": [genre["name"] for genre in movie_data.get("genres", [])],
                "spoken_languages": [lang["english_name"] for lang in movie_data.get("spoken_languages", [])],
                "production_countries": [country["name"] for country in movie_data.get("production_countries", [])],
            }
            tmdb_movies_list.append(movie_info)
        else:
            print(f"Error fetching data for {title}: Status code {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {title}: {e}")

# Get movie id

        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list
genres_list = [genre["name"] for genre in movie_data.get("genres", [])]

        # Extract the spoken_languages' English name into a list
spoken_languages_list = [lang["english_name"] for lang in movie_data.get("spoken_languages", [])]

        # Extract the production_countries' name into a list
production_countries_list = [country["name"] for country in movie_data.get("production_countries", [])]
print("Genres:", genres_list)
print("Spoken Languages:", spoken_languages_list)
print("Production Countries:", production_countries_list)

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found
print(f"Processed movie: {title}")

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(5):
    print(f"Result {i+1}:")
    print(json.dumps(tmdb_movies_list[i], indent=4))
    print("\n")

In [ ]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
print(df)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(nyt_reviews_df, tmdb_movies_df, on='title', how='inner')
print(merged_df)

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
   
    # Loop through characters to remove
    merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df.head())

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=['byline.person'])
print(merged_df)


In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
print(merged_df)